In [1]:
from __future__ import division
%matplotlib inline
# import, do some set up
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import keras as ks
from keras.layers import Dense
import tensorflow as tf
import os

dim = 10

def pca(data, dim = 4):
    nPCs = dim
    X = np.double(np.reshape(data , (data.shape[0]*data.shape[1] , data.shape[2])))
    mu = np.mean(X,axis=0)
    for i in range(data.shape[2]):
        X[:,i] = X[:,i] - mu[i]
    v = np.cov(X.T)
    eig_vals, eig_vecs = np.linalg.eig(v)
    img = np.zeros([data.shape[0], data.shape[1], dim], dtype=np.double)
    for i in range(nPCs):
        img[:,:,i] = np.reshape(np.matmul(X,eig_vecs[:,i]), (data.shape[0], data.shape[1]))
    return img

def reshape_to_input_format(input_):
    tmp = np.zeros([1,dim,1])
    for i in range(input_.shape[0]):
        tmp[0][i][0] = input_[i]
    return tmp

data = sio.loadmat('DataSet/Salinas.mat') # 512*217*204
gt = sio.loadmat('DataSet/Salinas_gt.mat')
X = data['salinas']
X = pca(X,dim)
#------- normalize data -----------
X -= np.amin(X)
X = X / np.amax(X)
X *= 255
X = np.int16(X)
# ---------------------------------
Y = gt['salinas_gt']
Y[Y==8] = 0
Y[Y==9] = 8
Y[Y==10] = 9
Y[Y==11] = 10
Y[Y==12] = 11
Y[Y==13] = 12
Y[Y==14] = 13
Y[Y==15] = 0
Y[Y==16] = 14

num_samples = 250
train = 200
test = 50
cols = dim
train_data = []
test_data = []
classk_train = {}
classk_test = {}
np.random.seed(1)
for k in range(1, np.amax(Y)+1):
    classk = X[Y==k, :]
    permutation_train = np.random.permutation(classk.shape[0])[:train]
    permutation_test = np.random.permutation(classk.shape[0])[train:num_samples]
    classk_train[k] = classk[permutation_train, :]
    classk_test[k] = classk[permutation_test, :]
    train_samples = np.zeros((train*(train-1), 1, cols, 1), dtype=np.uint8)
    test_samples = np.zeros((test*(test-1), 1, cols, 1), dtype=np.uint8)
    index = 0
    for i in range(train):
        for j in range(train):
            if i==j:
                continue
            train_data.append({"x":reshape_to_input_format(np.abs(classk_train[k][i, :] - classk_train[k][j, :])),"l":1})
            index += 1

    index = 0
    for i in range(test):
        for j in range(test):
            if i==j:
                continue
            test_data.append({"x":reshape_to_input_format(np.abs(classk_test[k][i, :] - classk_test[k][j, :])),"l":1})
            index += 1

classes = np.amax(Y)
n = int(train / (classes-1))
np.random.seed(1)
for i in range(classes):
    train_samples = np.zeros((train*n*(classes-1), 1, cols, 1), dtype=np.uint8)
    index = 0
    for j in range(classes):
        if i==j:
            continue
        temp_samples = classk_train[j+1][np.random.permutation(train)[:n], :]
        for k in range(train):
            for m in range(n):
                train_data.append({"x":reshape_to_input_format(np.abs(classk_train[i+1][k, :] - temp_samples[m, :])),"l":0})
                index += 1

n = int(test / (classes-1))
np.random.seed(1)
for i in range(classes):
    test_samples = np.zeros((test*n*(classes-1), 1, cols, 1), dtype=np.uint8)
    index = 0
    for j in range(classes):
        if i==j:
            continue
        temp_samples = classk_test[j+1][np.random.permutation(test)[:n], :]
        for k in range(test):
            for m in range(n):
                test_data.append({"x":reshape_to_input_format(np.abs(classk_test[i+1][k, :] - temp_samples[m, :])),"l":0})
                index += 1


print("train data size :  "+str(len(train_data)))
print("test data size :  "+str(len(test_data)))

np.random.shuffle(train_data)
np.random.shuffle(train_data)

np.random.shuffle(test_data)
np.random.shuffle(test_data)

train_x = []
train_y = []
test_x = []
test_y = []

for d in train_data:
    train_x.append(d['x'])
    train_y.append(d['l'])

for m in test_data:
    test_x.append(m['x'])
    test_y.append(m['l'])

train_x = np.asarray(train_x)
train_y = np.asarray(train_y)

test_x = np.asarray(test_x)
test_y = np.asarray(test_y)

Using TensorFlow backend.


train data size :  1103200
test data size :  61600


In [55]:
# model = ks.Sequential()

# model.add(ks.layers.Conv2D(3*dim, (1, 3), padding='same',activation='relu', input_shape=(1,dim,1)))
# model.add(ks.layers.Conv2D(9*dim, (1, 3),padding='same', activation='relu'))
# model.add(ks.layers.Dropout(0.30))
# model.add(ks.layers.Flatten())
# model.add(Dense(144, activation='relu'))
# model.add(Dense(72, activation='relu'))
# model.add(ks.layers.Dropout(0.5))
# model.add(Dense(36, activation='relu'))
# model.add(Dense(18, activation='relu'))
# model.add(Dense(6, activation='relu',kernel_regularizer = ks.regularizers.l1(0.01)))
# model.add(ks.layers.Dropout(0.35))
# model.add(Dense(1, activation='sigmoid'))

# sgd = ks.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# # rms = ks.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
# model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])


model = ks.Sequential()

model.add(ks.layers.Conv2D(30, (1, 3), padding='same',activation='relu', input_shape=(1,dim,1)))
model.add(ks.layers.Conv2D(60, (1, 3),padding='same', activation='relu'))
model.add(ks.layers.Conv2D(30, (1, 3), padding='same',activation='relu', strides=2))
model.add(ks.layers.Conv2D(10, (1, 3), padding='same',activation='relu',strides=2))
model.add(ks.layers.Dropout(0.30))
model.add(ks.layers.Flatten())
model.add(Dense(30, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu',kernel_regularizer = ks.regularizers.l1(0.01)))
model.add(ks.layers.Dropout(0.35))
model.add(Dense(1, activation='sigmoid'))

sgd = ks.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# rms = ks.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [56]:
model.fit(train_x, train_y,epochs=17, batch_size=256, verbose=1)

Epoch 1/17
1103200/1103200 [==============================] - 85s 77us/step - loss: 0.2753 - acc: 0.8996 1s - los
Epoch 2/17
1103200/1103200 [==============================] - 83s 75us/step - loss: 0.1989 - acc: 0.9269
Epoch 3/17
1103200/1103200 [==============================] - 84s 76us/step - loss: 0.1773 - acc: 0.9363
Epoch 4/17
1103200/1103200 [==============================] - ETA: 0s - loss: 0.1677 - acc: 0.940 - 82s 74us/step - loss: 0.1677 - acc: 0.9405
Epoch 5/17
1103200/1103200 [==============================] - 82s 75us/step - loss: 0.1615 - acc: 0.9428
Epoch 6/17
1103200/1103200 [==============================] - 82s 74us/step - loss: 0.1570 - acc: 0.9445
Epoch 7/17
1103200/1103200 [==============================] - 89s 81us/step - loss: 0.1546 - acc: 0.9451
Epoch 8/17
1103200/1103200 [==============================] - 91s 82us/step - loss: 0.1561 - acc: 0.9434
Epoch 9/17
1103200/1103200 [==============================] - 91s 82us/step - loss: 0.1536 - acc: 0.9444
Epoch 10

In [57]:
score = model.evaluate(test_x, test_y,verbose=1)
print(score)

61600/61600 [==============================] - 7s 106us/step
[0.15345417285507376, 0.9471428571428572]


In [58]:
model.save('mModel.h5')

In [59]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_85 (Conv2D)           (None, 1, 10, 30)         120       
_________________________________________________________________
conv2d_86 (Conv2D)           (None, 1, 10, 60)         5460      
_________________________________________________________________
conv2d_87 (Conv2D)           (None, 1, 5, 30)          5430      
_________________________________________________________________
conv2d_88 (Conv2D)           (None, 1, 3, 10)          910       
_________________________________________________________________
dropout_41 (Dropout)         (None, 1, 3, 10)          0         
_________________________________________________________________
flatten_21 (Flatten)         (None, 30)                0         
_________________________________________________________________
dense_75 (Dense)             (None, 30)                930       
__________